# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fd4f4d18bb0>
├── 'a' --> tensor([[-0.6265,  1.1939,  0.1541],
│                   [ 0.1267, -0.8559,  0.6731]])
└── 'x' --> <FastTreeValue 0x7fd4f4d185b0>
    └── 'c' --> tensor([[-0.6056,  2.2088,  1.0857,  0.0697],
                        [ 0.7068, -1.3729, -0.5308,  1.3769],
                        [ 0.2867,  0.6580,  0.5015,  0.0493]])

In [4]:
t.a

tensor([[-0.6265,  1.1939,  0.1541],
        [ 0.1267, -0.8559,  0.6731]])

In [5]:
%timeit t.a

63 ns ± 0.0311 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fd4f4d18bb0>
├── 'a' --> tensor([[-0.2427, -1.4583,  0.5375],
│                   [-0.8978,  0.7204, -0.1646]])
└── 'x' --> <FastTreeValue 0x7fd4f4d185b0>
    └── 'c' --> tensor([[-0.6056,  2.2088,  1.0857,  0.0697],
                        [ 0.7068, -1.3729, -0.5308,  1.3769],
                        [ 0.2867,  0.6580,  0.5015,  0.0493]])

In [7]:
%timeit t.a = new_value

65.7 ns ± 0.0554 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.6265,  1.1939,  0.1541],
               [ 0.1267, -0.8559,  0.6731]]),
    x: Batch(
           c: tensor([[-0.6056,  2.2088,  1.0857,  0.0697],
                      [ 0.7068, -1.3729, -0.5308,  1.3769],
                      [ 0.2867,  0.6580,  0.5015,  0.0493]]),
       ),
)

In [10]:
b.a

tensor([[-0.6265,  1.1939,  0.1541],
        [ 0.1267, -0.8559,  0.6731]])

In [11]:
%timeit b.a

57.1 ns ± 0.0391 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.7154,  0.3043, -0.5537],
               [ 0.7970,  0.7904, -1.2111]]),
    x: Batch(
           c: tensor([[-0.6056,  2.2088,  1.0857,  0.0697],
                      [ 0.7068, -1.3729, -0.5308,  1.3769],
                      [ 0.2867,  0.6580,  0.5015,  0.0493]]),
       ),
)

In [13]:
%timeit b.a = new_value

484 ns ± 0.0859 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

905 ns ± 7.06 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 26.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

135 µs ± 327 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 360 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fd448a1fa00>
├── 'a' --> tensor([[[-0.6265,  1.1939,  0.1541],
│                    [ 0.1267, -0.8559,  0.6731]],
│           
│                   [[-0.6265,  1.1939,  0.1541],
│                    [ 0.1267, -0.8559,  0.6731]],
│           
│                   [[-0.6265,  1.1939,  0.1541],
│                    [ 0.1267, -0.8559,  0.6731]],
│           
│                   [[-0.6265,  1.1939,  0.1541],
│                    [ 0.1267, -0.8559,  0.6731]],
│           
│                   [[-0.6265,  1.1939,  0.1541],
│                    [ 0.1267, -0.8559,  0.6731]],
│           
│                   [[-0.6265,  1.1939,  0.1541],
│                    [ 0.1267, -0.8559,  0.6731]],
│           
│                   [[-0.6265,  1.1939,  0.1541],
│                    [ 0.1267, -0.8559,  0.6731]],
│           
│                   [[-0.6265,  1.1939,  0.1541],
│                    [ 0.1267, -0.8559,  0.6731]]])
└── 'x' --> <FastTreeValue 0x7fd448a1f9d0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 193 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fd4f4d524f0>
├── 'a' --> tensor([[-0.6265,  1.1939,  0.1541],
│                   [ 0.1267, -0.8559,  0.6731],
│                   [-0.6265,  1.1939,  0.1541],
│                   [ 0.1267, -0.8559,  0.6731],
│                   [-0.6265,  1.1939,  0.1541],
│                   [ 0.1267, -0.8559,  0.6731],
│                   [-0.6265,  1.1939,  0.1541],
│                   [ 0.1267, -0.8559,  0.6731],
│                   [-0.6265,  1.1939,  0.1541],
│                   [ 0.1267, -0.8559,  0.6731],
│                   [-0.6265,  1.1939,  0.1541],
│                   [ 0.1267, -0.8559,  0.6731],
│                   [-0.6265,  1.1939,  0.1541],
│                   [ 0.1267, -0.8559,  0.6731],
│                   [-0.6265,  1.1939,  0.1541],
│                   [ 0.1267, -0.8559,  0.6731]])
└── 'x' --> <FastTreeValue 0x7fd4f4ce5460>
    └── 'c' --> tensor([[-0.6056,  2.2088,  1.0857,  0.0697],
                        [ 0.7068, -1.3729, -0.5308,  1.3769],
                 

In [23]:
%timeit t_cat(trees)

30.8 µs ± 67.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.9 µs ± 169 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.6265,  1.1939,  0.1541],
                [ 0.1267, -0.8559,  0.6731]],
       
               [[-0.6265,  1.1939,  0.1541],
                [ 0.1267, -0.8559,  0.6731]],
       
               [[-0.6265,  1.1939,  0.1541],
                [ 0.1267, -0.8559,  0.6731]],
       
               [[-0.6265,  1.1939,  0.1541],
                [ 0.1267, -0.8559,  0.6731]],
       
               [[-0.6265,  1.1939,  0.1541],
                [ 0.1267, -0.8559,  0.6731]],
       
               [[-0.6265,  1.1939,  0.1541],
                [ 0.1267, -0.8559,  0.6731]],
       
               [[-0.6265,  1.1939,  0.1541],
                [ 0.1267, -0.8559,  0.6731]],
       
               [[-0.6265,  1.1939,  0.1541],
                [ 0.1267, -0.8559,  0.6731]]]),
    x: Batch(
           c: tensor([[[-0.6056,  2.2088,  1.0857,  0.0697],
                       [ 0.7068, -1.3729, -0.5308,  1.3769],
                       [ 0.2867,  0.6580,  0.5015,  0.0493]],
         

In [26]:
%timeit Batch.stack(batches)

78.4 µs ± 148 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.6265,  1.1939,  0.1541],
               [ 0.1267, -0.8559,  0.6731],
               [-0.6265,  1.1939,  0.1541],
               [ 0.1267, -0.8559,  0.6731],
               [-0.6265,  1.1939,  0.1541],
               [ 0.1267, -0.8559,  0.6731],
               [-0.6265,  1.1939,  0.1541],
               [ 0.1267, -0.8559,  0.6731],
               [-0.6265,  1.1939,  0.1541],
               [ 0.1267, -0.8559,  0.6731],
               [-0.6265,  1.1939,  0.1541],
               [ 0.1267, -0.8559,  0.6731],
               [-0.6265,  1.1939,  0.1541],
               [ 0.1267, -0.8559,  0.6731],
               [-0.6265,  1.1939,  0.1541],
               [ 0.1267, -0.8559,  0.6731]]),
    x: Batch(
           c: tensor([[-0.6056,  2.2088,  1.0857,  0.0697],
                      [ 0.7068, -1.3729, -0.5308,  1.3769],
                      [ 0.2867,  0.6580,  0.5015,  0.0493],
                      [-0.6056,  2.2088,  1.0857,  0.0697],
                      [ 0.7068, -

In [28]:
%timeit Batch.cat(batches)

142 µs ± 74.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

307 µs ± 790 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
